In [ ]:
import tensorflow as tf

tf.__version__

In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.layers import (
    Dense,
    Input,
    Dropout,
    Flatten,
    Conv2D,
    BatchNormalization,
    Activation,
    MaxPooling2D,
)
from keras.optimizers import Adam

In [ ]:
tf.config.experimental.list_physical_devices("GPU")

In [ ]:
pathToLocal = "images/validation/"
pathToKaggle = "/kaggle/input/face-expression-recognition-dataset/images/validation/"

path = pathToLocal if os.path.exists(pathToLocal) else pathToKaggle

In [ ]:
plt.figure(figsize=(8, 8))
for i, expression in enumerate(path, start=1):
    img = load_img(f"images/validation/{expression}/{os.listdir(f'images/validation/{expression}')[0]}")
    plt.subplot(1, 7, i)
    plt.imshow(img)
    plt.title(expression)
    plt.axis("off")
plt.show()

# Check the amount of data in each folder in training dataset

In [ ]:
try:
    for expression in os.listdir("images/train/"):
        print(
            expression,
            "folder contains\t\t",
            len(os.listdir("images/train/" + expression)),
            "images",
        )
except:
    for expression in os.listdir(
            "/kaggle/input/face-expression-recognition-dataset/images/train/"
    ):
        print(
            expression,
            "folder contains\t\t",
            len(
                os.listdir(
                    "/kaggle/input/face-expression-recognition-dataset/images/train/"
                    + expression
                )
            ),
            "images",
        )

# Check the amount of data in each folder in testing dataset

In [ ]:
try:
    for expression in os.listdir("images/validation/"):
        print(
            expression,
            "folder contains\t\t",
            len(os.listdir("images/validation/" + expression)),
            "images",
        )
except:
    for expression in os.listdir(
            "/kaggle/input/face-expression-recognition-dataset/images/validation/"
    ):
        print(
            expression,
            "folder contains\t\t",
            len(
                os.listdir(
                    "/kaggle/input/face-expression-recognition-dataset/images/validation/"
                    + expression
                )
            ),
            "images",
        )

In [ ]:
model = tf.keras.models.Sequential(
    [
        Conv2D(64, (3, 3), padding="same", activation="relu", input_shape=(48, 48, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), padding="same", activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), padding="same", activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(512, (3, 3), padding="same", activation="relu"),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation="relu", kernel_regularizer=tf.keras.regularizers.l1()),
        Dropout(0.5),
        Dense(7, activation="softmax", kernel_regularizer=tf.keras.regularizers.l1()),
    ]
)

model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

# Data Augmentation

In [ ]:
try:
    datagen_train = ImageDataGenerator(
        rescale=1.0 / 255, zoom_range=0.3, horizontal_flip=True
    )

    train_generator = datagen_train.flow_from_directory(
        "/kaggle/input/face-expression-recognition-dataset/images/train/",
        batch_size=64,
        target_size=(48, 48),
        shuffle=True,
        color_mode="grayscale",
        class_mode="categorical",
    )

    datagen_test = ImageDataGenerator(
        rescale=1.0 / 255, zoom_range=0.3, horizontal_flip=True
    )

    test_generator = datagen_test.flow_from_directory(
        "/kaggle/input/face-expression-recognition-dataset/images/validation/",
        batch_size=64,
        target_size=(48, 48),
        shuffle=True,
        color_mode="grayscale",
        class_mode="categorical",
    )
except:
    datagen_train = ImageDataGenerator(
        rescale=1.0 / 255, zoom_range=0.3, horizontal_flip=True
    )

    train_generator = datagen_train.flow_from_directory(
        "images/train/",
        batch_size=64,
        target_size=(48, 48),
        shuffle=True,
        color_mode="grayscale",
        class_mode="categorical",
    )

    datagen_test = ImageDataGenerator(
        rescale=1.0 / 255, zoom_range=0.3, horizontal_flip=True
    )

    test_generator = datagen_test.flow_from_directory(
        "images/validation/",
        batch_size=64,
        target_size=(48, 48),
        shuffle=True,
        color_mode="grayscale",
        class_mode="categorical",
    )

steps_per_epoch = train_generator.n / train_generator.batch_size
testing_steps = test_generator.n / test_generator.batch_size

In [ ]:
train_generator = np.array(train_generator)
train_generator

In [ ]:
history = model.fit(x=train_generator, epochs=100, validation_split=0.2, batch_size=10)

In [ ]:
model.evaluate(x=test_generator, steps=testing_steps)

# Plot the training and validation accuracy and loss at each epoch

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, "y", label="Training loss")
plt.plot(epochs, val_loss, "r", label="Validation loss")
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
plt.plot(epochs, acc, label="Training acc")
plt.plot(epochs, val_acc, "r", label="Validation acc")
plt.title("Training and Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()